In [1]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du
from sklearn.externals import joblib

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")
# df_ratio = pd.read_csv("Feature_matrix/df_new_has_all_the_ratio_201808031807.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
df_train_cp = pd.read_csv("Feature_matrix/df_train_cp_201808062200.csv")

In [3]:
# import decode_utils as du
# import awesome_functions as af
from sklearn.metrics import confusion_matrix
from IPython.display import display, Markdown

def getAccuracy(y_true, y_pred, data_length, i):
    """
        y_true : 원래 타겟 컬럼의 데이터를 넣어준다.
        y_pred : 예측한 값을 넣어준다.
        data_length : 예측한 데이터의 총 개수를 넣어준다.
    """
    accuracy = round(np.trace(confusion_matrix(y_true, y_pred))/data_length, 4)
    display(Markdown("##### Accuracy : " + str(accuracy) + ", smoothing : " + str(i)))
    return accuracy

def fitNaiveBayesModel_smoothing(X, y, a = 1.0):
    return MultinomialNB(alpha=a).fit(X, y)

def __combine_dd_fl(df_dd, df_fl):
    col_we_dont_need = ["VisitNumber", "TripType", 'Return', 'Monday',\
                        'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    cols_we_need = [col for col in df_dd.columns if col not in col_we_dont_need]
    cols = [col for col in cols_we_need if col != "VisitNumber"]
    return du.concatDf(df_dd[cols], df_fl)
    
def get_concat_df_for_test_df(df_dd, df_fl):
    return __combine_dd_fl(df_dd, df_fl)
    
def get_df_has_specified_vn_list(vn_list, df=None, df_dd=None, df_fl=None, is_in=True,
                                 is_need_to_make_df = False):
    if is_need_to_make_df:
        df = __combine_dd_fl(df_dd, df_fl)
        print("Concat 완료.")
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def test_multinomial(df, early_stop = 5):
    X, y = af.get_df_to_fit(df)
    a = np.arange(0, 0.16, 0.01)
    accuracy_ = 0
    count = 0
    result = []
    alpha = []
    for i in a:
        if count >= early_stop:
            best_score = np.array(result).max()
            idx = result.index(best_score)
            display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
            return
        model_nb = fitNaiveBayesModel_smoothing(X, y, i)
        y_pred = model_nb.predict(X)
        accuracy = getAccuracy(y, y_pred, len(X), i)
        if accuracy_ > accuracy:
            count += 1
        accuracy_ = accuracy
        result.append(accuracy)
        alpha.append(i)
    best_score = np.array(result).max()
    idx = result.index(best_score)
    display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
#     return best_score, alpha[idx]

def cal_average_of_accuracy(li1, li2, score_li):    
    li3 = 96574 - len(li1 + li2)
    weight = [len(li1), len(li2), li3]
    result = 0
    for i, score in enumerate(score_li):
        result += weight[i] * score
    result = result/96574
    print(result)
    return result

In [4]:
def getSpecifiedVisitNumberData(df_train, vn):
    display(df_train[df_train.VisitNumber == vn])
#     display(df_train_dd[df_train_dd.VisitNumber == vn])

In [5]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [6]:
df_test_decoded = du.decodeStuffNeedsToBeDecoded(df_test)

In [7]:
tmp = df_decoded.groupby(["VisitNumber", "DepartmentDescription"]).agg({"ScanCount" : np.sum}).reset_index()

In [8]:
tmp.groupby(["VisitNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

,VisitNumber,DepartmentDescription,ScanCount
0,5,FINANCIAL SERVICES,-1
1,7,PERSONAL CARE,1
2,7,SHOES,1
3,8,DAIRY,2
4,8,DSD GROCERY,1
5,8,HOUSEHOLD CHEMICALS/SUPP,1
6,8,MEAT - FRESH & FROZEN,1
7,8,NULL,1
8,8,PAINT AND ACCESSORIES,18
9,8,PETS AND SUPPLIES,4


In [9]:
scan_total = tmp.groupby("VisitNumber").sum()["ScanCount"].reset_index(name='ScanCount')

In [10]:
sc_li = scan_total.ScanCount.values

In [11]:
cols_do_not_need = ['VisitNumber', 'Return', 'TripType', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']

In [30]:
df_train_cp[cols_do_not_need]

,VisitNumber,Return,TripType,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5,1,999,0,0,0,0,1,0,0
1,7,0,30,0,0,0,0,1,0,0
2,8,1,26,0,0,0,0,1,0,0
3,9,0,8,0,0,0,0,1,0,0
4,10,0,8,0,0,0,0,1,0,0
5,11,0,35,0,0,0,0,1,0,0
6,12,0,41,0,0,0,0,1,0,0
7,15,0,21,0,0,0,0,1,0,0
8,17,0,6,0,0,0,0,1,0,0
9,19,0,42,0,0,0,0,1,0,0


In [12]:
try_new_stuff = df_decoded.groupby(["Company", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [13]:
try_new_stuff_1 = try_new_stuff.groupby("Company").size().reset_index(name="Count")

In [14]:
cp_li = try_new_stuff_1[try_new_stuff_1.Count == 1].Company.unique()

In [15]:
len(cp_li)

4806

In [16]:
try_new_stuff = df_decoded.groupby(["FinelineNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [17]:
try_new_stuff_1 = try_new_stuff.groupby("FinelineNumber").size().reset_index(name="Count")

In [18]:
fl_li = try_new_stuff_1[try_new_stuff_1.Count == 1].FinelineNumber.unique()

In [19]:
len(fl_li)

2695

In [20]:
fl_li = fl_li.astype(float)

In [21]:
fl_li = fl_li.astype(str)

In [22]:
cp_fl = pd.concat([df_train_dd, df_train_fl[fl_li], df_train_cp[cp_li]], axis = 1)

In [23]:
cp_fl.head()

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,898999,899053,899157,899274,899407,899539,899675,899792,95,98
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def get_flat_type_user(df_, col = "DepartmentDescription", is_to_get_total = False):
    """
        예:) vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded) 반드시 df는 decoding 함수로 decoded된 것을 넣을 것.
    """
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] < 0, 0, df_count["Sc_sum"])
    df_count = df_count.dropna()
    df_count_total = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Total")
    if is_to_get_total:
        return df_count_total

    df_merged = pd.merge(df_count, df_count_total, on="VisitNumber")
    df_merged["P"] = df_merged["Sc_sum"].div(df_merged["Total"])
    
    num_category_li = []
    for i, vn in enumerate(df_merged.VisitNumber.unique()):
        tmp_df = df_merged[df_merged["VisitNumber"] == vn]
        if tmp_df.Total.values[0] == 0:
            num_category_li.append(0)
        else:
            num_of_category = len(tmp_df[col].unique())
            num_category_li.append(num_of_category)
        if i % 5000 == 0:
            print(str(i) + "까지 진행됨.")
    
    return num_category_li

In [25]:
num_fl = get_flat_type_user(df_decoded, "FinelineNumber")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [26]:
num_dd = get_flat_type_user(df_decoded)

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [28]:
cp_fl["N_DD"] = num_dd

In [29]:
cp_fl["N_FL"] = num_fl

In [30]:
num_company = get_flat_type_user(df_decoded, "Company")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [31]:
cp_fl["N_COMPANY"] = num_company

In [32]:
num_upc = get_flat_type_user(df_decoded, "Upc")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [33]:
cp_fl["N_UPC"] = 0

In [34]:
cp_fl.set_index("VisitNumber", inplace=True)

In [37]:
df_upc = df_decoded.groupby(["VisitNumber", "Upc"]).sum()["ScanCount"].reset_index(name="Sc_sum")

In [40]:
upc_vn_li = df_upc.groupby("VisitNumber").sum()["Sc_sum"].index.unique()

In [42]:
cp_fl.at[upc_vn_li,"N_UPC"] = num_upc

In [43]:
cp_fl = cp_fl.reset_index()

In [44]:
cp_fl["Scancount_total"] = sc_li

In [ ]:
cp_fl[fl_li] = cp_fl[fl_li].div(cp_fl["N_FL"])

In [ ]:
cp_fl[cp_li] = cp_fl[cp_li].div(cp_fl["N_COMPANY"])

In [ ]:
dd_li = [col for col in df_train_dd.columns if col not in cols_do_not_need]

In [ ]:
cp_fl[dd_li] = cp_fl[dd_li].div(cp_fl["N_DD"])

In [45]:
cp_fl.head()

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,899539,899675,899792,95,98,N_DD,N_FL,N_COMPANY,N_UPC,Scancount_total
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,7,17,15,21,28
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,3,3,3
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,3,3,3


In [46]:
from scipy.sparse import csr_matrix

In [47]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [48]:
train_X , train_y = af.get_df_to_fit(cp_fl)

In [49]:
label_enc = LabelEncoder().fit(train_y)
y_labeled = label_enc.transform(train_y)

In [50]:
train_X, test_X, train_y, test_y = train_test_split(train_X, y_labeled, random_state=0)

In [51]:
csr_train = csr_matrix(train_X.values)
csr_test = csr_matrix(test_X.values)

In [52]:
dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
num_boost_round = 1000
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':38,
          'max_depth':3,
          'eta': 0.25}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

[0]	train-mlogloss:2.09396	eval-mlogloss:2.09927
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:1.83211	eval-mlogloss:1.84236
[2]	train-mlogloss:1.66701	eval-mlogloss:1.68274
[3]	train-mlogloss:1.54389	eval-mlogloss:1.56293
[4]	train-mlogloss:1.44905	eval-mlogloss:1.47023
[5]	train-mlogloss:1.37333	eval-mlogloss:1.39599
[6]	train-mlogloss:1.31191	eval-mlogloss:1.33657
[7]	train-mlogloss:1.26195	eval-mlogloss:1.2887
[8]	train-mlogloss:1.21832	eval-mlogloss:1.24664
[9]	train-mlogloss:1.18222	eval-mlogloss:1.21181
[10]	train-mlogloss:1.15007	eval-mlogloss:1.18127
[11]	train-mlogloss:1.12279	eval-mlogloss:1.15483
[12]	train-mlogloss:1.09814	eval-mlogloss:1.13159
[13]	train-mlogloss:1.0757	eval-mlogloss:1.11111
[14]	train-mlogloss:1.05657	eval-mlogloss:1.09267
[15]	train-mlogloss:1.03871	eval-mlogloss:1.07594
[16]	train-mlogloss:1.02329	eval-mlogloss:1.0617
[17]	train-m

[156]	train-mlogloss:0.647644	eval-mlogloss:0.798175
[157]	train-mlogloss:0.646807	eval-mlogloss:0.797844
[158]	train-mlogloss:0.645985	eval-mlogloss:0.797562
[159]	train-mlogloss:0.645165	eval-mlogloss:0.797301
[160]	train-mlogloss:0.644323	eval-mlogloss:0.796896
[161]	train-mlogloss:0.643385	eval-mlogloss:0.796437
[162]	train-mlogloss:0.64252	eval-mlogloss:0.796141
[163]	train-mlogloss:0.641527	eval-mlogloss:0.795807
[164]	train-mlogloss:0.640807	eval-mlogloss:0.79551
[165]	train-mlogloss:0.639979	eval-mlogloss:0.795082
[166]	train-mlogloss:0.639205	eval-mlogloss:0.794866
[167]	train-mlogloss:0.638362	eval-mlogloss:0.794471
[168]	train-mlogloss:0.637557	eval-mlogloss:0.794117
[169]	train-mlogloss:0.636638	eval-mlogloss:0.793804
[170]	train-mlogloss:0.635756	eval-mlogloss:0.793426
[171]	train-mlogloss:0.634941	eval-mlogloss:0.793081
[172]	train-mlogloss:0.634108	eval-mlogloss:0.79282
[173]	train-mlogloss:0.633129	eval-mlogloss:0.792491
[174]	train-mlogloss:0.632236	eval-mlogloss:0.792

[312]	train-mlogloss:0.549746	eval-mlogloss:0.770514
[313]	train-mlogloss:0.549266	eval-mlogloss:0.770463
[314]	train-mlogloss:0.548833	eval-mlogloss:0.770375
[315]	train-mlogloss:0.548363	eval-mlogloss:0.770175
[316]	train-mlogloss:0.547943	eval-mlogloss:0.770057
[317]	train-mlogloss:0.547529	eval-mlogloss:0.769989
[318]	train-mlogloss:0.547102	eval-mlogloss:0.769898
[319]	train-mlogloss:0.546666	eval-mlogloss:0.769879
[320]	train-mlogloss:0.546246	eval-mlogloss:0.769753
[321]	train-mlogloss:0.545808	eval-mlogloss:0.769649
[322]	train-mlogloss:0.545434	eval-mlogloss:0.769574
[323]	train-mlogloss:0.544968	eval-mlogloss:0.769536
[324]	train-mlogloss:0.54455	eval-mlogloss:0.76944
[325]	train-mlogloss:0.544099	eval-mlogloss:0.769261
[326]	train-mlogloss:0.543594	eval-mlogloss:0.769133
[327]	train-mlogloss:0.543207	eval-mlogloss:0.769065
[328]	train-mlogloss:0.542815	eval-mlogloss:0.768997
[329]	train-mlogloss:0.542397	eval-mlogloss:0.768845
[330]	train-mlogloss:0.541964	eval-mlogloss:0.76

[468]	train-mlogloss:0.490927	eval-mlogloss:0.763219
[469]	train-mlogloss:0.490621	eval-mlogloss:0.763116
[470]	train-mlogloss:0.490282	eval-mlogloss:0.763074
[471]	train-mlogloss:0.489954	eval-mlogloss:0.763003
[472]	train-mlogloss:0.489662	eval-mlogloss:0.76295
[473]	train-mlogloss:0.489348	eval-mlogloss:0.762923
[474]	train-mlogloss:0.488991	eval-mlogloss:0.762918
[475]	train-mlogloss:0.488702	eval-mlogloss:0.76289
[476]	train-mlogloss:0.488393	eval-mlogloss:0.762879
[477]	train-mlogloss:0.488096	eval-mlogloss:0.762877
[478]	train-mlogloss:0.487827	eval-mlogloss:0.762881
[479]	train-mlogloss:0.487541	eval-mlogloss:0.762828
[480]	train-mlogloss:0.487252	eval-mlogloss:0.762829
[481]	train-mlogloss:0.48692	eval-mlogloss:0.76285
[482]	train-mlogloss:0.486604	eval-mlogloss:0.762875
[483]	train-mlogloss:0.486306	eval-mlogloss:0.762905
[484]	train-mlogloss:0.486002	eval-mlogloss:0.762906
[485]	train-mlogloss:0.485684	eval-mlogloss:0.762871
[486]	train-mlogloss:0.485374	eval-mlogloss:0.7628

In [55]:
import lightgbm

In [68]:
train_X = train_X.astype(float)
test_X = test_X.astype(float)

In [71]:
# X_train = csr_matrix(train_X.values)
# X_test = csr_matrix(test_X.values)

# dtrain = lightgbm.Dataset(X_train, label=train_y)
# dtest = lightgbm.Dataset(X_test, label=test_y)
num_boost_round = 4000
max_delta_step=0
learning_rate=0.008

params = {'objective':'multiclass',
          'boosting_type': 'gbdt',
          'max_depth' : -1,
          'nthread': 4,
          'metric': 'multi_logloss',
          'max_delta_step': 3, 
          'num_class':38,
          'learning_rate':learning_rate,
          'max_delta_step':max_delta_step,
          }

# evals = [(dtrain, 'train'), (dtest, 'test')]

# %%time
# 4000, 0.008
lightgbm_model = lightgbm.train(params = params,
                                train_set = dtrain, 
                                valid_sets = dtest,
                                num_boost_round = num_boost_round,
                                early_stopping_rounds=10)

[1]	valid_0's multi_logloss: 3.5578
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 3.48638
[3]	valid_0's multi_logloss: 3.42184
[4]	valid_0's multi_logloss: 3.36262
[5]	valid_0's multi_logloss: 3.30752
[6]	valid_0's multi_logloss: 3.25612
[7]	valid_0's multi_logloss: 3.20813
[8]	valid_0's multi_logloss: 3.16298
[9]	valid_0's multi_logloss: 3.12034
[10]	valid_0's multi_logloss: 3.07999
[11]	valid_0's multi_logloss: 3.04154
[12]	valid_0's multi_logloss: 3.00484
[13]	valid_0's multi_logloss: 2.96988
[14]	valid_0's multi_logloss: 2.93629
[15]	valid_0's multi_logloss: 2.9041
[16]	valid_0's multi_logloss: 2.87291
[17]	valid_0's multi_logloss: 2.84304
[18]	valid_0's multi_logloss: 2.81412
[19]	valid_0's multi_logloss: 2.7863
[20]	valid_0's multi_logloss: 2.75933
[21]	valid_0's multi_logloss: 2.73324
[22]	valid_0's multi_logloss: 2.70801
[23]	valid_0's multi_logloss: 2.68352
[24]	valid_0's multi_logloss: 2.65971
[25]	valid_0's multi_logloss: 2.63651


[213]	valid_0's multi_logloss: 1.24732
[214]	valid_0's multi_logloss: 1.2447
[215]	valid_0's multi_logloss: 1.24211
[216]	valid_0's multi_logloss: 1.23954
[217]	valid_0's multi_logloss: 1.23697
[218]	valid_0's multi_logloss: 1.23445
[219]	valid_0's multi_logloss: 1.23191
[220]	valid_0's multi_logloss: 1.22941
[221]	valid_0's multi_logloss: 1.22693
[222]	valid_0's multi_logloss: 1.22447
[223]	valid_0's multi_logloss: 1.22201
[224]	valid_0's multi_logloss: 1.21953
[225]	valid_0's multi_logloss: 1.21712
[226]	valid_0's multi_logloss: 1.21471
[227]	valid_0's multi_logloss: 1.2123
[228]	valid_0's multi_logloss: 1.20992
[229]	valid_0's multi_logloss: 1.20758
[230]	valid_0's multi_logloss: 1.20523
[231]	valid_0's multi_logloss: 1.20288
[232]	valid_0's multi_logloss: 1.20057
[233]	valid_0's multi_logloss: 1.19825
[234]	valid_0's multi_logloss: 1.19597
[235]	valid_0's multi_logloss: 1.1937
[236]	valid_0's multi_logloss: 1.19145
[237]	valid_0's multi_logloss: 1.18922
[238]	valid_0's multi_loglos

[423]	valid_0's multi_logloss: 0.942485
[424]	valid_0's multi_logloss: 0.941741
[425]	valid_0's multi_logloss: 0.94099
[426]	valid_0's multi_logloss: 0.940239
[427]	valid_0's multi_logloss: 0.939499
[428]	valid_0's multi_logloss: 0.93875
[429]	valid_0's multi_logloss: 0.938009
[430]	valid_0's multi_logloss: 0.937269
[431]	valid_0's multi_logloss: 0.936519
[432]	valid_0's multi_logloss: 0.9358
[433]	valid_0's multi_logloss: 0.935085
[434]	valid_0's multi_logloss: 0.934362
[435]	valid_0's multi_logloss: 0.933641
[436]	valid_0's multi_logloss: 0.932927
[437]	valid_0's multi_logloss: 0.932222
[438]	valid_0's multi_logloss: 0.931515
[439]	valid_0's multi_logloss: 0.930834
[440]	valid_0's multi_logloss: 0.930137
[441]	valid_0's multi_logloss: 0.929436
[442]	valid_0's multi_logloss: 0.928747
[443]	valid_0's multi_logloss: 0.928063
[444]	valid_0's multi_logloss: 0.927383
[445]	valid_0's multi_logloss: 0.92669
[446]	valid_0's multi_logloss: 0.926005
[447]	valid_0's multi_logloss: 0.92533
[448]	

[629]	valid_0's multi_logloss: 0.843709
[630]	valid_0's multi_logloss: 0.843419
[631]	valid_0's multi_logloss: 0.843131
[632]	valid_0's multi_logloss: 0.842857
[633]	valid_0's multi_logloss: 0.842569
[634]	valid_0's multi_logloss: 0.842284
[635]	valid_0's multi_logloss: 0.841997
[636]	valid_0's multi_logloss: 0.84172
[637]	valid_0's multi_logloss: 0.841437
[638]	valid_0's multi_logloss: 0.841145
[639]	valid_0's multi_logloss: 0.840863
[640]	valid_0's multi_logloss: 0.840586
[641]	valid_0's multi_logloss: 0.840296
[642]	valid_0's multi_logloss: 0.840021
[643]	valid_0's multi_logloss: 0.839758
[644]	valid_0's multi_logloss: 0.839472
[645]	valid_0's multi_logloss: 0.839203
[646]	valid_0's multi_logloss: 0.838921
[647]	valid_0's multi_logloss: 0.838635
[648]	valid_0's multi_logloss: 0.83835
[649]	valid_0's multi_logloss: 0.838064
[650]	valid_0's multi_logloss: 0.837801
[651]	valid_0's multi_logloss: 0.83753
[652]	valid_0's multi_logloss: 0.83726
[653]	valid_0's multi_logloss: 0.836978
[654

[835]	valid_0's multi_logloss: 0.803104
[836]	valid_0's multi_logloss: 0.802976
[837]	valid_0's multi_logloss: 0.802839
[838]	valid_0's multi_logloss: 0.802717
[839]	valid_0's multi_logloss: 0.802576
[840]	valid_0's multi_logloss: 0.802458
[841]	valid_0's multi_logloss: 0.802333
[842]	valid_0's multi_logloss: 0.802204
[843]	valid_0's multi_logloss: 0.802084
[844]	valid_0's multi_logloss: 0.801963
[845]	valid_0's multi_logloss: 0.801838
[846]	valid_0's multi_logloss: 0.801698
[847]	valid_0's multi_logloss: 0.80156
[848]	valid_0's multi_logloss: 0.801436
[849]	valid_0's multi_logloss: 0.801301
[850]	valid_0's multi_logloss: 0.801178
[851]	valid_0's multi_logloss: 0.801053
[852]	valid_0's multi_logloss: 0.800932
[853]	valid_0's multi_logloss: 0.800798
[854]	valid_0's multi_logloss: 0.800662
[855]	valid_0's multi_logloss: 0.800539
[856]	valid_0's multi_logloss: 0.80042
[857]	valid_0's multi_logloss: 0.800296
[858]	valid_0's multi_logloss: 0.800179
[859]	valid_0's multi_logloss: 0.800072
[8

...

[1241]	valid_0's multi_logloss: 0.772503
[1242]	valid_0's multi_logloss: 0.772462
[1243]	valid_0's multi_logloss: 0.772423
[1244]	valid_0's multi_logloss: 0.772387
[1245]	valid_0's multi_logloss: 0.772344
[1246]	valid_0's multi_logloss: 0.772309
[1247]	valid_0's multi_logloss: 0.772274
[1248]	valid_0's multi_logloss: 0.772233
[1249]	valid_0's multi_logloss: 0.772191
[1250]	valid_0's multi_logloss: 0.772146
[1251]	valid_0's multi_logloss: 0.772104
[1252]	valid_0's multi_logloss: 0.772065
[1253]	valid_0's multi_logloss: 0.772027
[1254]	valid_0's multi_logloss: 0.772002
[1255]	valid_0's multi_logloss: 0.771965
[1256]	valid_0's multi_logloss: 0.771932
[1257]	valid_0's multi_logloss: 0.771902
[1258]	valid_0's multi_logloss: 0.771855
[1259]	valid_0's multi_logloss: 0.771828
[1260]	valid_0's multi_logloss: 0.771791
[1261]	valid_0's multi_logloss: 0.771761
[1262]	valid_0's multi_logloss: 0.77173
[1263]	valid_0's multi_logloss: 0.771698
[1264]	valid_0's multi_logloss: 0.771664
[1265]	valid_0's 

[1442]	valid_0's multi_logloss: 0.766311
[1443]	valid_0's multi_logloss: 0.766286
[1444]	valid_0's multi_logloss: 0.766273
[1445]	valid_0's multi_logloss: 0.766252
[1446]	valid_0's multi_logloss: 0.766234
[1447]	valid_0's multi_logloss: 0.766209
[1448]	valid_0's multi_logloss: 0.766193
[1449]	valid_0's multi_logloss: 0.766172
[1450]	valid_0's multi_logloss: 0.766148
[1451]	valid_0's multi_logloss: 0.766123
[1452]	valid_0's multi_logloss: 0.766097
[1453]	valid_0's multi_logloss: 0.766071
[1454]	valid_0's multi_logloss: 0.76604
[1455]	valid_0's multi_logloss: 0.766013
[1456]	valid_0's multi_logloss: 0.765989
[1457]	valid_0's multi_logloss: 0.765961
[1458]	valid_0's multi_logloss: 0.76595
[1459]	valid_0's multi_logloss: 0.765916
[1460]	valid_0's multi_logloss: 0.765894
[1461]	valid_0's multi_logloss: 0.765872
[1462]	valid_0's multi_logloss: 0.765853
[1463]	valid_0's multi_logloss: 0.765824
[1464]	valid_0's multi_logloss: 0.765798
[1465]	valid_0's multi_logloss: 0.765778
[1466]	valid_0's m

[1643]	valid_0's multi_logloss: 0.762395
[1644]	valid_0's multi_logloss: 0.76239
[1645]	valid_0's multi_logloss: 0.762378
[1646]	valid_0's multi_logloss: 0.762362
[1647]	valid_0's multi_logloss: 0.76236
[1648]	valid_0's multi_logloss: 0.76235
[1649]	valid_0's multi_logloss: 0.76233
[1650]	valid_0's multi_logloss: 0.76231
[1651]	valid_0's multi_logloss: 0.762299
[1652]	valid_0's multi_logloss: 0.762292
[1653]	valid_0's multi_logloss: 0.762269
[1654]	valid_0's multi_logloss: 0.762247
[1655]	valid_0's multi_logloss: 0.762239
[1656]	valid_0's multi_logloss: 0.762233
[1657]	valid_0's multi_logloss: 0.762218
[1658]	valid_0's multi_logloss: 0.762199
[1659]	valid_0's multi_logloss: 0.762178
[1660]	valid_0's multi_logloss: 0.762166
[1661]	valid_0's multi_logloss: 0.762149
[1662]	valid_0's multi_logloss: 0.76213
[1663]	valid_0's multi_logloss: 0.762121
[1664]	valid_0's multi_logloss: 0.762104
[1665]	valid_0's multi_logloss: 0.762091
[1666]	valid_0's multi_logloss: 0.762079
[1667]	valid_0's multi

[1844]	valid_0's multi_logloss: 0.759986
[1845]	valid_0's multi_logloss: 0.759978
[1846]	valid_0's multi_logloss: 0.759964
[1847]	valid_0's multi_logloss: 0.759965
[1848]	valid_0's multi_logloss: 0.759959
[1849]	valid_0's multi_logloss: 0.75995
[1850]	valid_0's multi_logloss: 0.759938
[1851]	valid_0's multi_logloss: 0.759923
[1852]	valid_0's multi_logloss: 0.759913
[1853]	valid_0's multi_logloss: 0.759908
[1854]	valid_0's multi_logloss: 0.759902
[1855]	valid_0's multi_logloss: 0.759892
[1856]	valid_0's multi_logloss: 0.759883
[1857]	valid_0's multi_logloss: 0.759869
[1858]	valid_0's multi_logloss: 0.75986
[1859]	valid_0's multi_logloss: 0.759852
[1860]	valid_0's multi_logloss: 0.759835
[1861]	valid_0's multi_logloss: 0.759838
[1862]	valid_0's multi_logloss: 0.759828
[1863]	valid_0's multi_logloss: 0.759811
[1864]	valid_0's multi_logloss: 0.759803
[1865]	valid_0's multi_logloss: 0.7598
[1866]	valid_0's multi_logloss: 0.759799
[1867]	valid_0's multi_logloss: 0.759789
[1868]	valid_0's mul

[2045]	valid_0's multi_logloss: 0.758731
[2046]	valid_0's multi_logloss: 0.758723
[2047]	valid_0's multi_logloss: 0.758718
[2048]	valid_0's multi_logloss: 0.758715
[2049]	valid_0's multi_logloss: 0.758707
[2050]	valid_0's multi_logloss: 0.758705
[2051]	valid_0's multi_logloss: 0.758696
[2052]	valid_0's multi_logloss: 0.758691
[2053]	valid_0's multi_logloss: 0.758694
[2054]	valid_0's multi_logloss: 0.758697
[2055]	valid_0's multi_logloss: 0.758693
[2056]	valid_0's multi_logloss: 0.758692
[2057]	valid_0's multi_logloss: 0.758681
[2058]	valid_0's multi_logloss: 0.758675
[2059]	valid_0's multi_logloss: 0.758674
[2060]	valid_0's multi_logloss: 0.75867
[2061]	valid_0's multi_logloss: 0.758661
[2062]	valid_0's multi_logloss: 0.758659
[2063]	valid_0's multi_logloss: 0.758662
[2064]	valid_0's multi_logloss: 0.758657
[2065]	valid_0's multi_logloss: 0.758648
[2066]	valid_0's multi_logloss: 0.758655
[2067]	valid_0's multi_logloss: 0.758651
[2068]	valid_0's multi_logloss: 0.758652
[2069]	valid_0's 